<a href="https://colab.research.google.com/github/rohitvmeshram/Data-Craftsperson-Assessment/blob/main/Sales_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import sqlite3
import pandas as pd

In [37]:
import pandas as pd
import sqlite3

# Load the Excel file into a pandas DataFrame
df = pd.read_excel('/content/assessment_dataset.xlsx', engine='openpyxl')

# Check the first few rows of the DataFrame
print(df.head())

   TransactionID  CustomerID     TransactionDate  TransactionAmount  \
0              1     16795.0 2022-01-01 00:00:00            1910.91   
1              2      1860.0 2022-01-01 00:01:00           58590.27   
2              3     39158.0 2022-01-01 00:02:00             280.21   
3              4         NaN                 NaT           86485.63   
4              5     12284.0 2022-01-01 00:04:00             385.89   

  PaymentMethod  Quantity  DiscountPercent     City StoreType  CustomerAge  \
0          Cash         1            35.00  Kolkata  In-Store         32.0   
1          Cash         1             9.51  Lucknow    Online         40.0   
2    Debit Card         1            16.55  Lucknow    Online         52.0   
3           UPI         1             9.53   Jaipur  In-Store         62.0   
4    Debit Card        22             0.98  Kolkata    Online         31.0   

  CustomerGender  LoyaltyPoints ProductName Region Returned  FeedbackScore  \
0           Male          

In [38]:
import sqlite3

# Create a SQLite database in memory
conn = sqlite3.connect(':memory:')

# Load the DataFrame into the SQLite database
df.to_sql('df', conn, index=False, if_exists='replace')

500000

In [39]:
def run_query(query):
    return pd.read_sql_query(query, conn)

Total Sales and Average Discount per City

In [40]:
query1 = """
SELECT City,
       SUM(TransactionAmount) AS TotalSales,
       AVG(DiscountPercent) AS AvgDiscount
FROM df
GROUP BY City
ORDER BY TotalSales DESC;
"""
result1 = run_query(query1)
print("Total Sales and Average Order Value:")
print(result1)

Total Sales and Average Order Value:
        City    TotalSales  AvgDiscount
0    Kolkata  1.027326e+09    25.024795
1  Ahmedabad  1.023676e+09    25.055853
2  Bangalore  1.022380e+09    24.945146
3       Pune  1.022136e+09    24.981377
4    Chennai  1.022123e+09    24.897040
5      Delhi  1.021353e+09    25.009669
6    Lucknow  1.021208e+09    25.053024
7     Mumbai  1.018526e+09    25.066050
8     Jaipur  1.015044e+09    24.955417
9  Hyderabad  1.008892e+09    25.004120


 Top 5 Most Sold Products by Quantity

In [72]:
query2 = """
SELECT ProductName,
       SUM(Quantity) AS TotalQuantity
FROM df
GROUP BY ProductName
ORDER BY TotalQuantity DESC
LIMIT 5;

"""
result2 = run_query(query2)
print("Top 5 Most Sold Products by Quantity")
print(result2)

Top 5 Most Sold Products by Quantity
  ProductName  TotalQuantity
0       Apple        2296713
1        None         502299
2    Notebook         498649
3     T-Shirt         270545
4      Laptop          89809


Monthly Sales Trend Analysis

In [73]:
query3 = """
SELECT strftime('%Y-%m', TransactionDate) AS Month,
       SUM(TransactionAmount) AS TotalSales
FROM df
GROUP BY Month
ORDER BY Month;

"""
result3 = run_query(query3)
print("Monthly Sales Trend Analysis")
print(result3)

Monthly Sales Trend Analysis
      Month    TotalSales
0      None  1.024919e+09
1   2022-01  8.145187e+08
2   2022-02  7.441818e+08
3   2022-03  8.188742e+08
4   2022-04  8.018721e+08
5   2022-05  8.195627e+08
6   2022-06  7.904689e+08
7   2022-07  8.170296e+08
8   2022-08  8.221571e+08
9   2022-09  7.913500e+08
10  2022-10  8.154026e+08
11  2022-11  7.999454e+08
12  2022-12  3.423810e+08


Customer Segmentation by Age Groups

In [74]:
query4 = """
SELECT
    CASE
        WHEN CustomerAge BETWEEN 18 AND 25 THEN '18-25'
        WHEN CustomerAge BETWEEN 26 AND 35 THEN '26-35'
        WHEN CustomerAge BETWEEN 36 AND 45 THEN '36-45'
        WHEN CustomerAge BETWEEN 46 AND 60 THEN '46-60'
        ELSE '60+'
    END AS AgeGroup,
    COUNT(DISTINCT CustomerID) AS TotalCustomers
FROM df
GROUP BY AgeGroup
ORDER BY AgeGroup;

"""
result4 = run_query(query4)
print("Customer Segmentation by Age Groups:")
print(result4)

Customer Segmentation by Age Groups:
  AgeGroup  TotalCustomers
0    18-25           33619
1    26-35           37463
2    36-45           37490
3    46-60           43437
4      60+           46437


Average Transaction Amount by Payment Method

In [75]:
query5 = """
SELECT PaymentMethod,
       AVG(TransactionAmount) AS AvgTransactionAmount
FROM df
GROUP BY PaymentMethod
ORDER BY AvgTransactionAmount DESC;



"""
result5 = run_query(query5)
print("Average Transaction Amount by Payment Method")
print(result5)

Average Transaction Amount by Payment Method
  PaymentMethod  AvgTransactionAmount
0          Cash          22700.814183
1    Debit Card          22584.313091
2   Credit Card          22511.081923
3           UPI          22487.068090
4          None            914.664859


Top 3 Cities with Highest Product Returns

In [76]:
query6 = """SELECT City,
       COUNT(*) AS ReturnCount
FROM df
WHERE Returned = 'Yes'
GROUP BY City
ORDER BY ReturnCount DESC
LIMIT 3;


"""
result6 = run_query(query6)
print("Top 3 Cities with Highest Product Returns")
print(result6)

Top 3 Cities with Highest Product Returns
        City  ReturnCount
0      Delhi        25155
1  Bangalore        25126
2  Ahmedabad        25081


 Most Profitable Product (Based on Sales Revenue)

In [77]:
query7 = """
SELECT ProductName,
       SUM(TransactionAmount) AS TotalRevenue
FROM df
GROUP BY ProductName
ORDER BY TotalRevenue DESC
LIMIT 1;


"""
result7 = run_query(query7)
print("Most Profitable Product (Based on Sales Revenue)")
print(result7)

Most Profitable Product (Based on Sales Revenue)
  ProductName  TotalRevenue
0      Laptop  6.231220e+09


Percentage of Online vs In-Store Transactions

In [78]:
query8 = """
SELECT StoreType,
       COUNT(*) AS TotalTransactions,
       ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM df)), 2) AS Percentage
FROM df
GROUP BY StoreType;

"""
result8 = run_query(query8)
print("Percentage of Online vs In-Store Transactions")
print(result8)

Percentage of Online vs In-Store Transactions
  StoreType  TotalTransactions  Percentage
0      None              50000       10.00
1  In-Store             224782       44.96
2    Online             225218       45.04


Delivery Time Analysis (Average by Region)

In [79]:
query9 = """
SELECT Region,
       AVG(DeliveryTimeDays) AS AvgDeliveryTime
FROM df
GROUP BY Region
ORDER BY AvgDeliveryTime;


"""
result9 = run_query(query9)
print("Delivery Time Analysis (Average by Region)")
print(result9)

Delivery Time Analysis (Average by Region)
  Region  AvgDeliveryTime
0  North         4.981594
1   West         4.989695
2   East         5.051442
3  South         5.071966
4   None         7.484507


Find the total revenue (TransactionAmount) for each ProductName, but only for transactions where Returned is 'No' and FeedbackScore is greater than 3:
sql


In [80]:
query10 = """
SELECT
    ProductName,
    SUM(TransactionAmount) AS TotalRevenue
FROM df
WHERE Returned = 'No' AND FeedbackScore > 3
GROUP BY ProductName;


"""
result10 = run_query(query10)

print(result10)

  ProductName  TotalRevenue
0        None  9.097045e+06
1       Apple  4.477138e+06
2      Laptop  1.244789e+09
3    Notebook  4.815285e+06
4        Sofa  7.612135e+08
5     T-Shirt  2.061561e+07


 Find the top 3 customers (CustomerID) with the highest total spending (TransactionAmount) in each Region

In [81]:
query11 = """
WITH CustomerSpending AS (
    SELECT
        CustomerID,
        Region,
        SUM(TransactionAmount) AS TotalSpending,
        RANK() OVER (PARTITION BY Region ORDER BY SUM(TransactionAmount) DESC) AS Rank
    FROM df
    GROUP BY CustomerID, Region
)
SELECT
    CustomerID,
    Region,
    TotalSpending
FROM CustomerSpending
WHERE Rank <= 3;

"""
result11 = run_query(query11)

print(result11)

    CustomerID Region  TotalSpending
0          NaN   None   1.075674e+06
1      15328.0   None   8.469200e+03
2      31111.0   None   7.754860e+03
3          NaN   East   2.660439e+08
4      17142.0   East   4.185503e+05
5      26752.0   East   4.078095e+05
6          NaN  North   2.239016e+08
7      27065.0  North   4.159640e+05
8      28871.0  North   3.757432e+05
9          NaN  South   3.160311e+08
10     49018.0  South   5.053060e+05
11     21856.0  South   4.371897e+05
12         NaN   West   2.178666e+08
13     34314.0   West   4.109361e+05
14     10494.0   West   3.977506e+05


Calculate the percentage of returned transactions (Returned = 'Yes') for each ProductName:

In [82]:
query12 = """
SELECT
    ProductName,
    COUNT(*) AS TotalTransactions,
    SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) AS ReturnedTransactions,
    (SUM(CASE WHEN Returned = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS ReturnPercentage
FROM df
GROUP BY ProductName;

"""
result12 = run_query(query12)

print(result12)

  ProductName  TotalTransactions  ReturnedTransactions  ReturnPercentage
0        None              50000                 24990         49.980000
1       Apple              89970                 45033         50.053351
2      Laptop              89809                 44904         49.999443
3    Notebook              90294                 45061         49.904756
4        Sofa              89740                 44696         49.806107
5     T-Shirt              90187                 44783         49.655715


. Calculate the cumulative sum of TransactionAmount for each CustomerID ordered by TransactionDate:

In [83]:
query13 = """
SELECT
    CustomerID,
    TransactionDate,
    TransactionAmount,
    SUM(TransactionAmount) OVER (PARTITION BY CustomerID ORDER BY TransactionDate) AS CumulativeSum
FROM df;

"""
result13 = run_query(query13)

print(result13)

        CustomerID      TransactionDate  TransactionAmount  CumulativeSum
0              NaN                 None           86485.63   1.024919e+09
1              NaN                 None            1879.62   1.024919e+09
2              NaN                 None             384.35   1.024919e+09
3              NaN                 None             265.14   1.024919e+09
4              NaN                 None            1484.95   1.024919e+09
...            ...                  ...                ...            ...
499995     49999.0  2022-05-08 16:20:00           79335.10   7.956161e+04
499996     49999.0  2022-06-21 14:13:00           50987.95   1.305496e+05
499997     49999.0  2022-06-23 05:37:00             951.30   1.315009e+05
499998     49999.0  2022-08-19 15:18:00           75485.66   2.069865e+05
499999     49999.0  2022-09-17 07:03:00             436.74   2.074233e+05

[500000 rows x 4 columns]


Calculate the total revenue (TransactionAmount) for each CustomerGender, but only for transactions where CustomerAge is between 30 and 50:

In [84]:
query14 = """
SELECT
    CustomerGender,
    SUM(TransactionAmount) AS TotalRevenue
FROM df
WHERE CustomerAge BETWEEN 30 AND 50
GROUP BY CustomerGender;

"""
result14 = run_query(query14)

print(result14)

  CustomerGender  TotalRevenue
0         Female  1.252340e+09
1           Male  1.262418e+09
2          Other  1.241363e+09


 Identifying High-Value Customers (Customers with Highest Spending)

In [85]:
query15 = """
SELECT Region, SUM(ShippingCost) AS TotalShippingCost
FROM df
GROUP BY Region;


"""
result15 = run_query(query15)

print(result15)

  Region  TotalShippingCost
0   None       2.131938e+06
1   East       5.114604e+07
2  North       4.190867e+07
3  South       6.168813e+07
4   West       4.177538e+07


1. Total Sales and Average Discount per City
Insight: Identifies cities with the highest total sales and average discount percentages.

Usefulness: Helps businesses focus on high-performing cities and understand discount strategies in different regions.

2. Top 5 Most Sold Products by Quantity
Insight: Lists the top 5 products with the highest quantity sold.

Usefulness: Helps in inventory management and identifying popular products for marketing campaigns.

3. Monthly Sales Trend Analysis
Insight: Shows sales trends over time (monthly).

Usefulness: Helps in identifying seasonal trends, planning promotions, and forecasting future sales.

4. Customer Segmentation by Age Groups
Insight: Segments customers into age groups (e.g., 18-25, 26-35, etc.) and counts the number of customers in each group.

Usefulness: Helps in targeted marketing and understanding the demographics of the customer base.

5. Average Transaction Amount by Payment Method
Insight: Calculates the average transaction amount for each payment method (e.g., Cash, Debit Card, Credit Card, UPI).

Usefulness: Helps in understanding customer payment preferences and optimizing payment processing strategies.

6. Top 3 Cities with Highest Product Returns
Insight: Identifies cities with the highest number of returned products.

Usefulness: Helps in addressing return-related issues and improving customer satisfaction in those regions.

7. Most Profitable Product (Based on Sales Revenue)
Insight: Identifies the product generating the highest revenue.

Usefulness: Helps in focusing on high-revenue products and optimizing product offerings.

8. Percentage of Online vs In-Store Transactions
Insight: Shows the percentage of transactions conducted online versus in-store.

Usefulness: Helps in understanding customer preferences for online vs in-store shopping and optimizing sales channels.

9. Delivery Time Analysis (Average by Region)
Insight: Calculates the average delivery time for each region.

Usefulness: Helps in identifying regions with slower delivery times and improving logistics efficiency.

10. Total Revenue for Non-Returned Transactions with High Feedback
Insight: Calculates total revenue for products that were not returned and had a feedback score greater than 3.

Usefulness: Helps in identifying high-quality products that contribute positively to revenue and customer satisfaction.

11. Top 3 Customers with Highest Spending in Each Region
Insight: Identifies the top 3 customers with the highest spending in each region.

Usefulness: Helps in recognizing high-value customers for loyalty programs and personalized marketing.

12. Percentage of Returned Transactions by Product
Insight: Calculates the percentage of returned transactions for each product.

Usefulness: Helps in identifying products with high return rates and addressing potential quality or customer satisfaction issues.

13. Cumulative Sum of Transaction Amount by Customer
Insight: Shows the cumulative spending of each customer over time.

Usefulness: Helps in understanding customer lifetime value and identifying loyal customers.

14. Total Revenue by Customer Gender for Specific Age Group
Insight: Calculates total revenue for transactions where the customer's age is between 30 and 50, segmented by gender.

Usefulness: Helps in understanding the spending patterns of a specific demographic group for targeted marketing.

15. Total Shipping Cost by Region
Insight: Calculates the total shipping cost for each region.

Usefulness: Helps in identifying regions with high shipping costs and optimizing logistics strategies.

Overall Usefulness of the Analysis
Data-Driven Decision Making: The analysis provides actionable insights that can guide business decisions, such as optimizing marketing strategies, improving customer satisfaction, and enhancing operational efficiency.

Customer Insights: By segmenting customers based on age, gender, and spending behavior, businesses can tailor their offerings to meet the needs of different customer groups.

Product Performance: Identifying top-selling and high-return products helps in inventory management and product quality improvement.

Regional Analysis: Understanding sales, returns, and delivery times by region allows businesses to address region-specific challenges and opportunities.

Trend Identification: Monthly sales trends and cumulative spending patterns help in forecasting and strategic planning.